Research for the statistical analysis of meteorological data and machine learning methods for determining solar irradiation.

Researcher: Matheus Henrique da Silva.
Control and Automation Engineering Student.
Universidade Tecnológica Federal do Paraná, Cornélio Procópio, Paraná, Brasil. 
E-mail: matheussilva.2019@alunos.utfpr.edu.br. 
ID Lattes: 5450995625966991.

Supervisor: Wesley Angelino de Souza.
Lecturer in the Graduate Program in Electrical Engineering.
Universidade Tecnológica Federal do Paraná, Cornélio Procópio, Paraná, Brasil. 
E-mail: wesleyangelino@utfpr.edu.br. 
ID Lattes: 8594457321079718.

---
---
Importing Libraries

In [ ]:
import folium

---
---

#### Step 1 - data extraction and agglutination

Data collected and made available by the Instituto Nacional de Meteorologia (INMET), acess on https://portal.inmet.gov.br/, in format .csv of 606 meteorological collection stations of Brazil between 2010 and 2021.

Each .csv file contains 9 lines with information about the station: Nome; Codigo Estacao; Latitude; Longitude; Altitude; Situacao; Data Inicial; Data Final e Periodicidade da Medicao.

In line 11, contains the name of the variables: Data Medicao; Hora Medicao; PRECIPITACAO TOTAL, HORARIO(mm); PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA(mB); PRESSAO ATMOSFERICA REDUZIDA NIVEL DO MAR, AUT(mB); PRESSAO ATMOSFERICA MAX.NA HORA ANT. (AUT)(mB); PRESSAO ATMOSFERICA MIN. NA HORA ANT. (AUT)(mB); RADIACAO GLOBAL(Kj/mÂ²); TEMPERATURA DA CPU DA ESTACAO(Â°C); TEMPERATURA DO AR - BULBO SECO, HORARIA(Â°C); TEMPERATURA DO PONTO DE ORVALHO(Â°C); TEMPERATURA MAXIMA NA HORA ANT. (AUT)(Â°C); TEMPERATURA MINIMA NA HORA ANT. (AUT)(Â°C); TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT)(Â°C); TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT)(Â°C); TENSAO DA BATERIA DA ESTACAO(V); UMIDADE REL. MAX. NA HORA ANT. (AUT)(%); UMIDADE REL. MIN. NA HORA ANT. (AUT)(%); UMIDADE RELATIVA DO AR, HORARIA(%); VENTO, DIRECAO HORARIA (gr)(Â° (gr)); VENTO, RAJADA MAXIMA(m/s); VENTO, VELOCIDADE HORARIA(m/s).

From line 12, the data collected.


In [ ]:
# Dataframe with the informations of all stations

df_stations = pd.DataFrame(columns =  ["csv", "nome", "codigo", "latitude", 
                                       "longitude", "altitude", "situacao", "data_inicio", 
                                       "data_fim", "periodicidade"])

for file in glob.glob("*.csv"):
        
    arquivo = file
    file = open(file)

    csv_reader = csv.reader(file)

    # 1: 'nome': nome_estacao
    row = next(csv_reader)
    row=str(row[0])
    tmp=row.split(": ")
    nome_estacao = tmp[1]
    
    # 2: 'codigo': codigo_estacao
    row = next(csv_reader)
    row=str(row[0])
    tmp=row.split(": ")
    codigo_estacao = tmp[1]
    
    # 3: 'latitude': latitude_estacao
    row = next(csv_reader)
    row=str(row[0])
    tmp=row.split(": ")
    latitude_estacao = tmp[1]
    
    # 4: 'longitude': longitude_estacao
    row = next(csv_reader)
    row=str(row[0])
    tmp=row.split(": ")
    longitude_estacao = tmp[1]
    
    # 5: 'altitude': altitude_estacao
    row = next(csv_reader)
    row=str(row[0])
    tmp=row.split(": ")
    altitude_estacao = tmp[1]
    
    # 5: 'situacao': situacao_estacao
    row = next(csv_reader)
    row=str(row[0])
    tmp=row.split(": ")
    situacao_estacao = tmp[1]
    
    # 6: 'data_inicio': data_inicio_coleta_estacao
    row = next(csv_reader)
    row=str(row[0])
    tmp=row.split(": ")
    data_inicio_coleta_estacao = tmp[1]
    
    # 7: 'data_fim': data_fim_coleta_estacao
    row = next(csv_reader)
    row=str(row[0])
    tmp=row.split(": ")
    data_fim_coleta_estacao = tmp[1]

    # 8: 'periodicidade': peridiocidade_dados_estacao
    row = next(csv_reader)
    row=str(row[0])
    tmp=row.split(": ")
    peridiocidade_dados_estacao = tmp[1]
    
    new_row = {'csv': str(arquivo), 
               'nome': nome_estacao, 
               'codigo': codigo_estacao, 
               'latitude': latitude_estacao,
               'longitude': longitude_estacao,
               'altitude': altitude_estacao,
               'situacao': situacao_estacao, 
               'data_inicio': data_inicio_coleta_estacao, 
               'data_fim': data_fim_coleta_estacao, 
               'periodicidade': peridiocidade_dados_estacao}
    
    df_stations = df_stations.append(new_row, ignore_index=True)

In [ ]:
df_stations.head()

In [ ]:
# Identification of data types and changes for station selection

data_types = df_stations.dtypes

print(data_types)

df_stations['latitude'] = df_stations['latitude'].astype(float)
df_stations['longitude'] = df_stations['longitude'].astype(float)

In [ ]:
# Selection of the stations in Cornélio Procópio-PR region

lat = -23.185038698153438
lon = -50.647548006591066
dist = 4

lim_lat_higher = lat + dist 
lim_lat_bottom = lat - dist

lon =  -50.647548006591066
lim_lon_right = lon + dist
lim_lon_left = lon - dist

selected_stations = df_stations[((df_stations['latitude'] >= lim_lat_bottom) & (df_stations['latitude'] <= lim_lat_higher)) & \
                            ((df_stations['longitude'] >= lim_lon_left) & (df_stations['longitude'] <= lim_lon_right))]

In [ ]:
selected_stations.head()

In [ ]:
# Dynamic map for station location

map = folium.Map(location=[lat, lon], zoom_start=14)

folium.Marker(location=[lat, lon], popup='CORNÉLIO PROCÓPIO').add_to(map)

lenght = selected_stations.shape[0]

lat_ar = np.zeros(lenght)
lon_ar = np.zeros(lenght)
est_ar = np.empty((lenght), dtype=object)

index = selected_stations.index.to_numpy()

for i in range(lenght):

    lat_ar[i] = selected_stations.loc[index[i], 'latitude']
    lon_ar[i] = selected_stations.loc[index[i], 'longitude']
    est_ar[i] = selected_stations.loc[index[i], 'nome']

    popup_station = folium.Popup(f'{est_ar[i]}\n{lat_ar[i]},{lon_ar[i]}')
    folium.Marker(location=[lat_ar[i], lon_ar[i]], popup=popup_station).add_to(map)

map

In [ ]:
# Dataframe with the data agglutination

data_loaded = 1

if(data_loaded==0):
    
    result = pd.DataFrame()

    for file in selected_stations['csv']:

        df_temp = pd.read_csv(file, skiprows=10, delimiter=";", decimal=",")
        df_temp.drop(df_temp.columns[[4,5,6,8,11,12,13,14,15,16,17,22]],axis=1, inplace = True)
        df_temp.dropna(inplace=True)

        result = pd.concat([result, df_temp])
        
    result.to_csv('data_agglutinated/data_selected_stations.csv') 
    
else:
    result = pd.read_csv('data_agglutinated/data_selected_stations.csv', delimiter=",")

In [ ]:
result.head()

---
---

#### Step 2 - statistical and exploratory analysis


##### Step 2.1 - load and transformation


In [ ]:
# Load data

data = pd.read_csv('data_agglutinated/data_selected_stations.csv')

In [ ]:
# Change variables name and format of date

data.drop(data.columns[0],axis=1, inplace = True)
data.rename(columns = {'Data Medicao': 'data'}, inplace = True)
data['data'] = pd.to_datetime(data['data'])
data.rename(columns = {'Hora Medicao': 'hora'}, inplace = True)
data.rename(columns = {'PRECIPITACAO TOTAL, HORARIO(mm)': 'precipitacao'}, inplace = True)
data.rename(columns = {'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA(mB)': 'pressao_atm'}, inplace = True)
data.rename(columns = {'RADIACAO GLOBAL(Kj/m²)': 'radiacao'}, inplace = True)
data.rename(columns = {'TEMPERATURA DO AR - BULBO SECO, HORARIA(°C)': 'temperatura'}, inplace = True)
data.rename(columns = {'TEMPERATURA DO PONTO DE ORVALHO(°C)': 'temp_orvalho'}, inplace = True)
data.rename(columns = {'UMIDADE RELATIVA DO AR, HORARIA(%)': 'umidade'}, inplace = True)
data.rename(columns = {'VENTO, DIRECAO HORARIA (gr)(° (gr))': 'direcao_vento'}, inplace = True)
data.rename(columns = {'VENTO, RAJADA MAXIMA(m/s)': 'vento_maximo'}, inplace = True)
data.rename(columns = {'VENTO, VELOCIDADE HORARIA(m/s)': 'velocidade'}, inplace = True)

In [ ]:
# Scatter plot of target variable 'radiacao' by 'hora'

data.plot(kind="scatter", x="hora", y="radiacao", alpha=0.1)

plt.savefig('figures/scatter_radiacao_hora.png', format='png', dpi = 500)

In the preliminary analyses, we noticed that the data are concentrated between 10 am and 10 pm

However, solar radiation begins to be noticed along with sunrise ~ 6:40 am

Therefore, we will shift the data by -3 hours each (due to the Brazilian time zone)

In [ ]:
def adjust_hour(df):

    df['hora'] = df['hora'].replace(0, 21)
    df['hora'] = df['hora'].replace(100, 22)
    df['hora'] = df['hora'].replace(200, 23)
    df['hora'] = df['hora'].replace(300, 0)
    df['hora'] = df['hora'].replace(400, 1)
    df['hora'] = df['hora'].replace(500, 2)
    df['hora'] = df['hora'].replace(600, 3)
    df['hora'] = df['hora'].replace(700, 4)
    df['hora'] = df['hora'].replace(800, 5)
    df['hora'] = df['hora'].replace(900, 6)
    df['hora'] = df['hora'].replace(1000, 7)
    df['hora'] = df['hora'].replace(1100, 8)
    df['hora'] = df['hora'].replace(1200, 9)
    df['hora'] = df['hora'].replace(1300, 10)
    df['hora'] = df['hora'].replace(1400, 11)
    df['hora'] = df['hora'].replace(1500, 12)
    df['hora'] = df['hora'].replace(1600, 13)
    df['hora'] = df['hora'].replace(1700, 14)
    df['hora'] = df['hora'].replace(1800, 15)
    df['hora'] = df['hora'].replace(1900, 16)
    df['hora'] = df['hora'].replace(2000, 17)
    df['hora'] = df['hora'].replace(2100, 18)
    df['hora'] = df['hora'].replace(2200, 19)
    df['hora'] = df['hora'].replace(2300, 20)

    return df

In [ ]:
data = adjust_hour(data)

In [ ]:
# data dataframe info, quantities and types

data.info()

In [ ]:
# Checking for NULL values

non_null_counts = data.count()

print(non_null_counts)

In [ ]:
# Statistical describe

data.describe()

##### Step 2.2 - data cleaning

Previously, there were discrepant values ​​noticed at unconventional times, for this reason they caused the outliers to be cleaned in a different way:

We take the data per hour, and remove according to the variable_target 'radiacao' the last 3.5 * standard deviation


In [ ]:
def remove_out(df):

    dados_0 = df[df['hora'] == 0]
    dados_1 = df[df['hora'] == 1]
    dados_2 = df[df['hora'] == 2]
    dados_3 = df[df['hora'] == 3]
    dados_4 = df[df['hora'] == 4]
    dados_5 = df[df['hora'] == 5]
    dados_6 = df[df['hora'] == 6]
    dados_7 = df[df['hora'] == 7]
    dados_8 = df[df['hora'] == 8]
    dados_9 = df[df['hora'] == 9]
    dados_10 = df[df['hora'] == 10]
    dados_11 = df[df['hora'] == 11]
    dados_12 = df[df['hora'] == 12]
    dados_13 = df[df['hora'] == 13]
    dados_14 = df[df['hora'] == 14]
    dados_15 = df[df['hora'] == 15]
    dados_16 = df[df['hora'] == 16]
    dados_17 = df[df['hora'] == 17]
    dados_18 = df[df['hora'] == 18]
    dados_19 = df[df['hora'] == 19]
    dados_20 = df[df['hora'] == 20]
    dados_21 = df[df['hora'] == 21]
    dados_22 = df[df['hora'] == 22]
    dados_23 = df[df['hora'] == 23]

    data_hour = [dados_0, dados_1, dados_2, dados_3, dados_4, dados_5, dados_6, dados_7, dados_8, dados_9, dados_10, dados_11, dados_12, dados_13, dados_14, dados_15, dados_16, dados_17, dados_18, dados_19, dados_20, dados_21, dados_22, dados_23]

    data_filtered = pd.DataFrame()

    for i in data_hour:

        mean_radiacao = np.mean(i['radiacao'])
        std_radiacao = np.std(i['radiacao'])

        lower = mean_radiacao - 3.5 * std_radiacao
        upper = mean_radiacao + 3.5 * std_radiacao

        y = i.query('radiacao <= @upper')

        data_filtered = pd.concat([data_filtered, y])

    return data_filtered

In [ ]:
data = remove_out(data)

In [ ]:
# Scatter plot of target variable 'radiacao' by 'hora' with filtered data

data.plot(kind="scatter", x="hora", y="radiacao", alpha=0.1)

plt.savefig('figures/scatter_radiacao_hora_filtered.png', format='png', dpi = 500)

##### Step 2.3 - statistical analysis


In [ ]:
# Statistical describe with filtered data

data.describe()

In [ ]:
# Histogram of the variables

def histogram_variables(df, filename=None):

    df.hist(bins=50, figsize=(20,15))

    if filename is not None:
        plt.savefig(filename, format='pdf', bbox_inches='tight')

    plt.show()

In [ ]:
histogram_variables(data, 'figures/histogram_variables.pdf')

In [ ]:
# Scatter of the target variable 'radiacao' with all variables

def scatter_variables(df, variable, filename=None):

    variables = df.columns.drop(variable)

    data = df[[variable] + list(variables)]

    sns.pairplot(data, x_vars=variables, y_vars=variable, height=2.5)

    plt.tight_layout()

    if filename is not None:
        plt.savefig(filename, dpi=500, bbox_inches='tight')

    plt.show()

In [ ]:
scatter_variables(data, 'radiacao', 'figures/scatter_variables.png')

In [ ]:
# Skewness values

data.skew(axis = 0)

In [ ]:
# Pearson correlation

def pearson_correlation_map(df, filename=None):

    plt.figure(figsize = (12,8))

    sns.heatmap(df.corr(), annot = True, fmt = '.4f', cmap = 'Reds', vmax = .99, vmin = -0.60)

    if filename is not None:
        plt.savefig(filename, format='pdf', bbox_inches='tight')

    plt.show()

In [ ]:
pearson_correlation_map(data, 'figures/pearson_correlation_map.pdf')

---
---

#### Step 3 - data normalization
